### IMPORTAZIONE LIBRERIE E DATASET

In [12]:
import pandas as pd
import zipfile
import numpy as np
import nltk
import string
import time
nltk.download('stopwords')
from nltk.corpus import stopwords
from itertools import combinations
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Notebook\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

zip_file_path = "adoptions/dog.zip"
csv_file_name = "dogs.csv"
zip_file_path2 = "adoptions/dogtravel.zip"
csv_file_name2 = "dogTravel.csv"

with zipfile.ZipFile(zip_file_path) as zip_file:
    with zip_file.open(csv_file_name) as csv_file:
        df = pd.read_csv(csv_file)
        
with zipfile.ZipFile(zip_file_path2) as zip_file:
    with zip_file.open(csv_file_name2) as csv_file:
        df2 = pd.read_csv(csv_file)
        
cnt = pd.read_csv('countries.csv')

In [14]:
#df.to_csv("prova1.csv")

### DATA CLEANING 

MODIFICA DATE

In [16]:
# Uso una regex per identificare le date e le NON DATE
is_date = df['posted'].str.contains('\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}\+\d{4}')

is_date = is_date.reset_index(drop=True)
not_date = ~is_date
#creo due dataset
not_date = df[~is_date]
is_date=df[is_date]
#creo una copia di not_date
new_not_date = not_date.copy()
#sposto i valori di una colonna a destra
new_not_date['accessed'] = new_not_date['stateQ']
new_not_date['stateQ'] = new_not_date['contact_country']
new_not_date['contact_country'] = new_not_date['contact_zip']
new_not_date['contact_zip'] = new_not_date['contact_state']
new_not_date['contact_state'] = new_not_date['contact_city']
new_not_date['contact_city'] = new_not_date['posted']
new_not_date['posted'] = new_not_date['status']
#assegno adoptable a tutte le righe di status
new_not_date['status']='adoptable'
# Unione di is_date e new_not_date
new = pd.concat([is_date, new_not_date], axis=0)
# Riordino le righe in base all'indice del dataframe di partenza
new = new.reindex(index=df.index)
new.shape
df=new

MODIFICA STATEQ

In [17]:
#IMPORTO COUNTRIES 
cnt = pd.read_csv('countries.csv')
cnt = cnt.astype(str)

state_groups = cnt.groupby('STATE')
#creo un dizionario vuoto
state_zips = {}

# loop sugli stati
for state_name, state_group in state_groups:
    # estraggo gli zip code
    state_zips[state_name] = state_group['ZIP'].tolist()


In [18]:
import re

zip_codes = [zip_code for zips in state_zips.values() for zip_code in zips]
#regex per identificare i numeri a 5 cifre
num_regex = re.compile(r'^\d{5}$')

# Ciclo sui valori di StateQ
for i, state in enumerate(df['stateQ'].astype(str).str.strip().tolist()):
    # controllo che stateq è un numero a 5 cifre ed è presente in zip_codes
    if num_regex.match(state) and state in zip_codes:
        # trova il valore nel dizionario
        for key, value in state_zips.items():
            if state in value:
                # sostituisce il valore
                df.at[i, 'stateQ'] = key

In [19]:
#df.head()

## 1. Extract all dogs with status that is not adoptable¶


In [20]:
is_not_adottable=df['status'] != 'adoptable'
non_adottabili=df[is_not_adottable]
non_adottabili

,id,org_id,url,type.x,species,breed_primary,breed_secondary,breed_mixed,breed_unknown,color_primary,color_secondary,color_tertiary,age,sex,size,coat,fixed,house_trained,declawed,special_needs,shots_current,env_children,env_dogs,env_cats,name,status,posted,contact_city,contact_state,contact_zip,contact_country,stateQ,accessed,type.y,description,stay_duration,stay_cost


In [21]:
print("Non ci sono status non adoptable, le date al posto dei valori adoptable sono stati rimossi nel processo di data cleaning e sono state sostituite con il valore piu frequente della colonna: adoptable, che aveva un numero di record di 58147 sul totale di 58180")
#
#

Non ci sono status non adoptable, le date al posto dei valori adoptable sono stati rimossi nel processo di data cleaning e sono state sostituite con il valore piu frequente della colonna: adoptable, che aveva un numero di record di 58147 sul totale di 58180


## 2. For each (primary) breed, determine the number of dogs¶

In [22]:
breeds = df.groupby('breed_primary').size()
breeds.shape

(216,)

In [23]:
breeds.head(10)

breed_primary
Affenpinscher                  17
Afghan Hound                    4
Airedale Terrier               19
Akbash                          3
Akita                         181
Alaskan Malamute               72
American Bulldog             1134
American Eskimo Dog            43
American Foxhound              17
American Hairless Terrier       4
dtype: int64

## 3. For each (primary) breed, determine the ratio between the number of dogs of `Mixed Breed` and those not of Mixed Breed. Hint: look at the `secondary_breed`.

In [24]:
is_mixed=(df["breed_mixed"]==True)|(df["breed_primary"]=="Mixed Breed")
mixed=df[is_mixed]
colmix=mixed["breed_primary"]
is_not_mixed= (df["breed_mixed"]==False)&(df["breed_primary"]!="Mixed Breed")
not_mixed=df[is_not_mixed]
colnmix=not_mixed["breed_primary"]

### calcolo del ratio 

In [25]:
colmix_counts = colmix.value_counts()
colnmix_counts = colnmix.value_counts()

ratio_data = []
for breed, count in df['breed_primary'].value_counts().items():
    colmix_count = colmix_counts.get(breed, 0)
    colnmix_count = colnmix_counts.get(breed, 0)
    
    if colnmix_count == 0:
        ratio = float('inf')
    else:
        ratio = round(colmix_count / colnmix_count, 2)
        
    ratio_data.append({
        'breed': breed,
        'mixed_count': colmix_count,
        'non_mixed_count': colnmix_count,
        'ratio': ratio
    })

ratio = pd.DataFrame(ratio_data)
ratio = ratio.sort_values("breed")
ratio.shape

(216, 4)

In [26]:
ratio.head()

,breed,mixed_count,non_mixed_count,ratio
127,Affenpinscher,5,12,0.42
177,Afghan Hound,4,0,inf
121,Airedale Terrier,17,2,8.50
185,Akbash,2,1,2.00
49,Akita,83,98,0.85


## 4. For each (primary) breed, determine the earliest and the latest `posted` timestamp.

In [27]:
#converto le date in formato datetime64
df.dtypes
df['posted'] = pd.to_datetime(df['posted'])
df.dtypes
#uso groupby
breed_posted_range = df.groupby('breed_primary')['posted'].agg(['min', 'max'])
result = breed_posted_range.rename(columns={'min': 'Earliest','max':'Latest'})

result.head()

,Earliest,Latest
breed_primary,,
Affenpinscher,2012-03-08 10:27:33+00:00,2019-09-14 10:10:51+00:00
Afghan Hound,2017-06-29 23:28:51+00:00,2019-07-27 00:38:48+00:00
Airedale Terrier,2014-06-13 12:59:36+00:00,2019-09-19 18:40:39+00:00
Akbash,2019-07-21 00:35:59+00:00,2019-08-23 17:11:04+00:00
Akita,2012-03-03 09:31:08+00:00,2019-09-20 15:19:57+00:00


## 5. For each state, compute the sex imbalance, that is the difference between male and female dogs. In which state this imbalance is largest?

In [28]:
### HO USATO LA TECNICA DELLO SMOOTHING 

In [29]:
# raggruppo per stato e per sesso ed estraggo i valori di sesso con .unstack
gender_diff = df.groupby('stateQ')['sex'].value_counts().unstack().fillna(0)
# applico la tecnica di smoothing aggiungendo 1 a entrambi i numeratori e i denominatori
gender_diff['Female'] = gender_diff['Female'] + 1
gender_diff['Male'] = gender_diff['Male'] + 1
# calcolo la differenza
gender_diff['diff'] = gender_diff['Male'] / gender_diff['Female']
# elimino la tabella Unknown (contiene solo 3 valori)
gender_diff = gender_diff.drop('Unknown', axis=1)
#inverto le colonne male e female
gender_diff = gender_diff.reindex(columns=['Male', 'Female', 'diff'])

gender_diff.head(10)


sex,Male,Female,diff
stateQ,,,
AK,9.0,8.0,1.125000
AL,510.0,535.0,0.953271
AR,345.0,352.0,0.980114
AZ,1143.0,1028.0,1.111868
CA,533.0,497.0,1.072435
CO,862.0,913.0,0.944140
CT,3443.0,3289.0,1.046823
DC,2451.0,2220.0,1.104054
DE,1124.0,1245.0,0.902811


In [30]:
#trovo la massima differenza
max_diff = gender_diff['diff'].max()
#creo una maschere boleana
mask = gender_diff['diff'] == max_diff
#assegno il risultato al dataframe state_eith_max_diff
state_with_max_diff = gender_diff[mask]
state_with_max_diff

sex,Male,Female,diff
stateQ,,,
MD,27.0,11.0,2.454545


#### il Maryland è lo stato col sex imbalance piu alto

## 6. For each pair (age, size), determine the average duration of the stay and the average cost of stay.


In [31]:
#converto stay duration a numerico
df['stay_duration'] = pd.to_numeric(df['stay_duration'], errors='coerce')
df['stay_cost'] = pd.to_numeric(df['stay_cost'], errors='coerce')

In [32]:
#non ci sono valori=0
is_null_value=df["stay_cost"].isnull()
null=df[is_null_value]
null.head()

,id,org_id,url,type.x,species,breed_primary,breed_secondary,breed_mixed,breed_unknown,color_primary,color_secondary,color_tertiary,age,sex,size,coat,fixed,house_trained,declawed,special_needs,shots_current,env_children,env_dogs,env_cats,name,status,posted,contact_city,contact_state,contact_zip,contact_country,stateQ,accessed,type.y,description,stay_duration,stay_cost


In [33]:
#CI SONO 198 VALORI DI STAY COST MINORI DI ZERO
is_minore_di_zero=df["stay_cost"]<0
minori=df[is_minore_di_zero]
minori.shape


(198, 37)

In [34]:
#elimino le righe con valori minori di zero
df.drop(minori.index, inplace=True)
df.shape

(57982, 37)

In [35]:
# Calcolo per ogni coppia (age, size) la mediana di stay_duration e stay_cost 
result = df.groupby(['age', 'size'])[['stay_duration', 'stay_cost']].agg('median')

print(result)

                    stay_duration  stay_cost
age    size                                 
Adult  Extra Large           88.0    224.120
       Large                 90.0    229.230
       Medium                89.0    229.160
       Small                 89.0    230.670
Baby   Extra Large           87.0    236.050
       Large                 90.0    230.275
       Medium                90.0    230.390
       Small                 90.0    231.435
Senior Extra Large           90.0    229.650
       Large                 89.0    230.060
       Medium                89.0    232.145
       Small                 89.0    232.195
Young  Extra Large           90.0    246.570
       Large                 90.0    230.660
       Medium                89.0    231.280
       Small                 90.0    230.825


## 7. Find the dogs involved in at least 3 travels. Also list the breed of those dogs.

In [36]:
# CONSIDERO IL NUM DI VALORI DIVERSI PER PAESE (COLONNA FOUND DI DOGTRAVEL)

In [37]:
df2.head()

,index,id,contact_city,contact_state,description,found,manual,remove,still_there
0,0,44520267,Anoka,MN,Boris is a handsome mini schnauzer who made hi...,Arkansas,NaN,NaN,NaN
1,1,44698509,Groveland,FL,Duke is an almost 2 year old Potcake from Abac...,Abacos,Bahamas,NaN,NaN
2,2,45983838,Adamstown,MD,Zac Woof-ron is a heartthrob movie star lookin...,Adam,Maryland,NaN,NaN
3,3,44475904,Saint Cloud,MN,~~Came in to the shelter as a transfer from an...,Adaptil,NaN,True,NaN
4,4,43877389,Pueblo,CO,Palang is such a sweetheart. She loves her peo...,Afghanistan,NaN,NaN,NaN


In [38]:
#df2=pd.read_csv('adoptions/dogTravel.csv')
# conto il numero di valori unici di found e raggruppo per id
# in questo modo escludo i valori duplicati, per ogni id ho un unico found
grouped = df2.groupby('id')['found'].nunique()
# seleziono solo gli id dove found è >= a 3
filtered = grouped[grouped >= 3]
#rinomino le colonne 
result = filtered.reset_index().rename(columns={'found': 'viaggi'})[['id', 'viaggi']]
#faccio il Merge con df
df3 = pd.merge(df, result, how='inner')
#mostro 3 colonne : id , razza e numero di viaggi
viaggi=df3[["id", "name","breed_primary","viaggi"]]
#infine controllo che id non abbia duplicati
viaggi["id"].duplicated().any()

False

In [39]:
viaggi.shape

(447, 4)

In [40]:
#mostro il risultato in ordine decrescente di viaggi
viaggi.sort_values('viaggi',ascending=False).head(10)

,id,name,breed_primary,viaggi
220,45537987,Lakota Mato (Bear),Alaskan Malamute,7
264,44070616,Finn MacCool,Alaskan Malamute,7
208,45728583,Murry Murdoch da Tank!,Alaskan Malamute,7
251,44572953,Apollo the White Knight,Alaskan Malamute,7
44,45988387,Winter Vale,Alaskan Malamute,7
271,36978896,"Maddie \Cutie Patootie!\"",""Maddie is our littl...",Alaskan Malamute,7
272,33218331,"Bucky \Are u my hooman?\"",""Born in August 2014...",Alaskan Malamute,7
401,45670129,TJ,Papillon,5
391,45860138,Ladybird,Redbone Coonhound,5
371,45991032,Pinto,Labrador Retriever,5


## 8. Fix the travels table so that the correct state is computed from the manual and the found fields. If manual is not missing, then it overrides what is stored in found.

In [41]:
df2.head()

,index,id,contact_city,contact_state,description,found,manual,remove,still_there
0,0,44520267,Anoka,MN,Boris is a handsome mini schnauzer who made hi...,Arkansas,NaN,NaN,NaN
1,1,44698509,Groveland,FL,Duke is an almost 2 year old Potcake from Abac...,Abacos,Bahamas,NaN,NaN
2,2,45983838,Adamstown,MD,Zac Woof-ron is a heartthrob movie star lookin...,Adam,Maryland,NaN,NaN
3,3,44475904,Saint Cloud,MN,~~Came in to the shelter as a transfer from an...,Adaptil,NaN,True,NaN
4,4,43877389,Pueblo,CO,Palang is such a sweetheart. She loves her peo...,Afghanistan,NaN,NaN,NaN


In [42]:
df2['found'] = df2['manual'].where(df2['manual'].notnull(), df2['found'])
df2.head()

,index,id,contact_city,contact_state,description,found,manual,remove,still_there
0,0,44520267,Anoka,MN,Boris is a handsome mini schnauzer who made hi...,Arkansas,NaN,NaN,NaN
1,1,44698509,Groveland,FL,Duke is an almost 2 year old Potcake from Abac...,Bahamas,Bahamas,NaN,NaN
2,2,45983838,Adamstown,MD,Zac Woof-ron is a heartthrob movie star lookin...,Maryland,Maryland,NaN,NaN
3,3,44475904,Saint Cloud,MN,~~Came in to the shelter as a transfer from an...,Adaptil,NaN,True,NaN
4,4,43877389,Pueblo,CO,Palang is such a sweetheart. She loves her peo...,Afghanistan,NaN,NaN,NaN


## 9. For each state, compute the ratio between the number of travels and the population.

In [43]:
## nel dataframe df2 ogni valore di found con un valore di id diverso rappresenta un viaggio
## trovare il numero di viaggi per ogni stato
##matchare col dataframe NST_EST2021

In [44]:
grouped = df2.groupby('found')['id'].nunique()

In [45]:
result = grouped.reset_index().rename(columns={'id': 'viaggi','found':'stato'})[['stato', 'viaggi']]
pop=pd.read_csv("adoptions/NST-EST2021-POP.csv", header=None)
pop.columns = ['stato', 'population']

In [46]:
#tengo solo i risultati in comune e droppo i 2 non in comune
mask = pop['stato'].isin(result['stato'])
pop = pop[mask]
#print(pop)

In [47]:
#MEGING DELLA COLONNA VIAGGI E CALCOLO DEL RATIO
merged = pd.merge(pop, result, on='stato')
#print(merged)
merged['viaggi'] = pd.to_numeric(merged['viaggi'])
merged['population'] = pd.to_numeric(merged['population'].str.replace('.', '', regex=True))
merged['viaggi/population'] = merged['viaggi'] / merged['population']
merged.head()

,stato,population,viaggi,viaggi/population
0,Alabama,5024279,325,0.000065
1,Arizona,7151502,32,0.000004
2,Arkansas,3011524,55,0.000018
3,California,39538223,138,0.000003
4,Colorado,5773714,7,0.000001


## 10. For each dog, compute the number of days from the posted day to the day of last access.

In [48]:
#controllo se ci sono cani ripetuti(id ripetuti)

In [49]:
#non ci sono duplicati
is_duplicate=df["id"].duplicated()
duplicated=df[is_duplicate]
duplicated.shape

(0, 37)

In [50]:
#converto i valori in formato datetime
df['posted'] = pd.to_datetime(df['posted']).dt.date
df['accessed'] = pd.to_datetime(df['accessed']).dt.date

In [51]:
#calcolo la differenza tra posted e accessed
time_diff = df['accessed'] - df['posted']
#converto la differenza in giorni
days_diff = time_diff.dt.days
# aggiungo la colonna DaysDiffs al dataframe
df['DaysDiff'] = days_diff
result=df[['id','posted','accessed','DaysDiff']].copy()

MOSTRO I RISULTATI IN ORDINE DECRESCENTE

In [52]:
result.sort_values("DaysDiff", ascending=False).head()

,id,posted,accessed,DaysDiff
5781,604115,2003-05-18,2019-09-20,5969
5780,2592031,2004-02-23,2019-09-20,5688
55505,2613506,2004-02-29,2019-09-20,5682
34231,4527948,2005-05-30,2019-09-20,5226
44815,5142790,2005-09-26,2019-09-20,5107


## 11. Partition the dogs according to the number of weeks from the posted day to the day of last access.

PER OGNI RECORD MOSTRO IL NUMERO DI SETTIMANE TRASCORSE

In [53]:
result['settimane'] = result['DaysDiff'] // 7
groups = result.groupby('settimane')
new_df = groups.apply(lambda x: x)

In [54]:
new_df.head()

,id,posted,accessed,DaysDiff,settimane
0,46042150,2019-09-20,2019-09-20,0,0
1,46042002,2019-09-20,2019-09-20,0,0
2,46040898,2019-09-20,2019-09-20,0,0
3,46039877,2019-09-20,2019-09-20,0,0
4,46039306,2019-09-20,2019-09-20,0,0


PER OGNI SETTIMANA MOSTRO IL NUMERO DI RECORD

In [55]:
# Creo una nuova colonna chiamata "settimane" 
result['settimane'] = result['DaysDiff'] // 7
# Raggruppo i cani in base al numero di settimane trascorse
groups = result.groupby('settimane')
new_df = groups.size().reset_index(name='num_cani')
new_df = new_df.set_index('settimane')

In [56]:
new_df.head()

,num_cani
settimane,
0,9765
1,6530
2,5745
3,3341
4,2434


## 12. Find for duplicates in the dogs dataset. Two records are duplicates if they have (1) same breeds and sex, and (2) they share at least 90% of the words in the description field. Extra points if you find and implement a more refined for determining if two rows are duplicates.

TROVO I CANI CHE HANNO STESSA RAZZA E STESSO SESSO E CONVERTO DESCRIPTION IN STRINGA

In [57]:
df2=df.sort_values(by=['breed_primary', 'sex'])

In [58]:
df2['description'] = df2['description'].astype(str)

ELIMINO LE DESCRIZIONI VUOTE

In [59]:
df2['description'].replace('nan', np.nan, inplace=True)
print(df2['description'].isnull().sum())

8676


In [60]:
df2.dropna(subset=['description'], inplace=True)
df2.shape

(49306, 38)

TEXT-PREPROCESSING

ELIMINO I LINK

In [61]:
#1 RIMUOVO I LINK
import re
# Define a regular expression pattern for finding links in text
pattern = r'(https?://[^\s]+)|(\bwww\.[^\s]+)'
def remove_links(text):
    # Use the regular expression pattern to find and remove links in the text
    text_without_links = re.sub(pattern, '', text)
    return text_without_links
# Apply the remove_links function to the 'description' column of the dataframe
df2['description'] = df2['description'].apply(remove_links)

RIMUOVO LA PUNTEGGIATURA E PORTO IN LOWERCASE 

In [62]:
def clean_text(text):
    # Porta il testo in lowercase
    text = text.lower()
    # Rimuove la punteggiatura
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)
    return text
df2['description'] = df2['description'].apply(clean_text)

RIMUOVO LE STOPWORDS

In [63]:
stop_words = set(stopwords.words('english'))

# Funzione per rimuovere le stopwords dal testo
def remove_stopwords(text):
    # Dividiamo il testo in parole
    words = text.split()
    # Rimuoviamo le stopwords
    filtered_words = [word for word in words if word.lower() not in stop_words]
    # Ricomponiamo il testo senza le stopwords
    text_without_stopwords = ' '.join(filtered_words)
    return text_without_stopwords

# Applichiamo la funzione di rimozione delle stopwords alla colonna "description" del DataFrame
df2['description'] = df2['description'].apply(remove_stopwords)

In [64]:
#PRENDO UNO SLICE DI DF3 
df4=df2.copy()
df4=df4[["id","name","breed_primary","sex","description"]]
#df4.to_csv('leven/df4.csv')
#print(df4.iloc[92])
df4.drop(df4[df4['sex'] == 'Unknown'].index, inplace=True)

In [65]:
df4.shape

(49303, 5)

### UTILIZZO LA LIBRERIA sklearn, in particolare utilizzo:

* CountVectorizer * : questa classe viene utilizzata per creare la rappresentazione vettoriale delle descrizioni dei cani.

* cosine_similarity* : questa funzione calcola la similarità coseno tra le righe di una matrice. 

La variabile X è una matrice densa (ossia una matrice di numeri reali in cui ogni cella contiene un valore) ottenuta dalla rappresentazione vettoriale delle descrizioni. La dimensione di questa matrice sarà pari al numero di descrizioni per il numero di parole nel dizionario.

La variabile sim_matrix, invece, è una matrice quadrata (ossia una matrice con lo stesso numero di righe e colonne) che rappresenta la similarità coseno tra ogni coppia di descrizioni presenti nella matrice X. La dimensione di questa matrice sarà pari al numero di descrizioni per il numero di descrizioni.
 
Sim_indices conterrà quindi una lista di indici di descrizioni simili per ogni descrizione presente nel DataFrame.

la variabile coppia è una tupla che contiene gli indici di due descrizioni simili. La similarità tra queste due descrizioni viene quindi calcolata utilizzando l'indice corrispondente nella matrice sim_matrix e viene memorizzata come valore della chiave 'similarity' nella variabile coppia_complete.


In [66]:
def generate_couples(subset):
    # creazione della rappresentazione vettoriale delle descrizioni
    descrizioni = subset['description'].tolist()
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(descrizioni)

    # calcolo della similarità coseno tra le descrizioni
    sim_matrix = cosine_similarity(X)

    # ottenimento degli indici delle descrizioni simili
    sim_indices = pd.DataFrame(sim_matrix, columns=subset.index, index=subset.index)
    sim_indices = sim_indices.apply(lambda x: x[x >= 0.9].index.tolist(), axis=1)
    sim_indices = sim_indices[sim_indices.apply(len) > 1]

    # creazione delle coppie complete
    coppie_complete = []
    coppie_viste = set()
    for _, ids in sim_indices.items():
        for coppia in combinations(ids, 2):
            # verifica se la coppia è già stata vista
            if coppia in coppie_viste:
                continue
            #ULTERIORE CONTROLLO PER CONSIDERARE SOLO LE COPPIE CON SIMILARITò >=90    
            if sim_matrix[subset.index.get_loc(coppia[0]), subset.index.get_loc(coppia[1])] >= 0.9:
                coppia_complete = {'id1': subset.loc[coppia[0], 'id'],
                       'id2': subset.loc[coppia[1], 'id'],
                       'name1': subset.loc[coppia[0], 'name'],
                       'name2': subset.loc[coppia[1], 'name'],
                       'descrizione1': subset.loc[coppia[0], 'description'],
                       'descrizione2': subset.loc[coppia[1], 'description'],
                       'breed_primary1': subset.loc[coppia[0], 'breed_primary'],
                       'breed_primary2': subset.loc[coppia[1], 'breed_primary'],
                       'sex1': subset.loc[coppia[0], 'sex'],
                       'sex2': subset.loc[coppia[1], 'sex'],
                       'similarity': sim_matrix[subset.index.get_loc(coppia[0]), subset.index.get_loc(coppia[1])]
                      }
                coppie_complete.append(coppia_complete)
                coppie_viste.add(coppia)
    return coppie_complete

def find_pairs_with_groups(cani):
    # raggruppa gli id per sesso e razza
    groups = cani.groupby(['sex', 'breed_primary'])['id'].apply(set)

    coppie_complete = []
    for group in groups:
        # filtra i cani del gruppo
        subset = cani.loc[cani['id'].isin(group)]
        coppie_complete.extend(generate_couples(subset))

    return coppie_complete


In [67]:
cani2=df4.copy()
cani2 = cani2.reset_index(drop=True)
#cani2=cani2.head(20000)
cani2.shape

(49303, 5)

In [68]:
start_time = time.time()

result2=find_pairs_with_groups(cani2)

end_time = time.time()
execution_time = end_time - start_time
print(f"Tempo di esecuzione: {execution_time} secondi")

Tempo di esecuzione: 28.0184965133667 secondi


In [69]:
resultdef=pd.DataFrame(result2)
resultdef.shape

(9281, 11)

In [70]:
resultdef.head(10)

,id1,id2,name1,name2,descrizione1,descrizione2,breed_primary1,breed_primary2,sex1,sex2,similarity
0,45970614,45871731,Elsie,Sidney,elsiejpgthank looking available pets cats felv...,55351jpgthank looking available pets cats felv...,Affenpinscher,Affenpinscher,Female,Female,0.990826
1,46023964,46023963,MAYA,LEAH,adoption fee fully sponsored adopt pay 000,adoption fee fully sponsored adopt pay 000,Alaskan Malamute,Alaskan Malamute,Female,Female,1.000000
2,45990095,45989321,191216 Ranga,191585 Mia,ranga three month old pup came us stray puppy ...,mia looks year old around 40 lbs timid girl fi...,American Bulldog,American Bulldog,Female,Female,0.928593
3,45897570,45897563,Mercury,Neptune,pretty little girl lives foster home enjoys sn...,neptune lives foster home enjoys snuggles play...,American Bulldog,American Bulldog,Female,Female,0.935971
4,45897570,45872595,Mercury,Jupiter,pretty little girl lives foster home enjoys sn...,precious jupiter lives foster home enjoys snug...,American Bulldog,American Bulldog,Female,Female,0.920751
5,45897563,45872595,Neptune,Jupiter,neptune lives foster home enjoys snuggles play...,precious jupiter lives foster home enjoys snug...,American Bulldog,American Bulldog,Female,Female,0.950947
6,43170920,42087185,Mollie,Molly,dogs adopted troy animal rescue project date v...,chickens cats dogs adopted troy animal rescue ...,American Bulldog,American Bulldog,Female,Female,0.971825
7,45639449,45639446,Lola Jewel,Lola Jewel,lola jewel jewel foster says 3yrs old guesstim...,lola jewel jewel foster says 3yrs old guesstim...,American Bulldog,American Bulldog,Female,Female,1.000000
8,45255573,45252099,Cheyenne - Courtesy Listing,Cheyenne - Courtesy Listing,people inquiring pets listed courtesy listings...,people inquiring pets listed courtesy listings...,American Bulldog,American Bulldog,Female,Female,1.000000
9,44518435,44517341,Tyla,Tyla,meet tyla adorable american bulldog mix 411 yo...,meet tyla adorable american bulldog mix 411 yo...,American Bulldog,American Bulldog,Female,Female,1.000000


In [102]:
#resultdef.sort_values('similarity',ascending=True).head(100)

## 12 bonus. SE LE DESCRIZIONI HANNO SIMILARITA' <1 CONTROLLO I NOMI.

Questo controllo mi permette di filtrare i valori che non hanno descrizioni identiche ma si riferiscono allo stesso cane. 

In [73]:
# Creazione della maschera booleana per selezionare le righe con similarità < 1
mask1 = resultdef['similarity'] < 0.99

# Creazione della maschera booleana per selezionare le righe con lo stesso nome
mask2 = resultdef['name1'] == resultdef['name2']

# Combinazione delle maschere booleane con l'operatore logico &
mask = mask1 & mask2

# Applicazione della maschera booleana al DataFrame
result_filtered = resultdef[mask]


In [74]:
result_filtered.shape

(111, 11)

In [75]:
result_filtered.sort_values('similarity',ascending=True).head()

,id1,id2,name1,name2,descrizione1,descrizione2,breed_primary1,breed_primary2,sex1,sex2,similarity
6751,38278645,28082621,JR,JR,jr funny little boy desperately needs furever ...,jr funny little boy desparately needs furever ...,Jack Russell Terrier,Jack Russell Terrier,Male,Male,0.902109
2064,45008032,45003858,Rosalind B sweet soul,Rosalind B sweet soul,please contact vickie dvbrooks57gmailcom infor...,meet sweet rosalind sweet soul little timid be...,Labrador Retriever,Labrador Retriever,Female,Female,0.902973
104,45758135,44077006,dinky,dinky,check youtube update stough little girl nothin...,little dinky came us weighing barely 15 lbs ea...,American Staffordshire Terrier,American Staffordshire Terrier,Female,Female,0.904250
5928,45572628,37073161,Mac,Mac,mac loyal friend life around 78 years old gorg...,mac loyal friend life around 78 years old gorg...,Cocker Spaniel,Cocker Spaniel,Male,Male,0.904599
7274,42564378,42549629,Gunner B lower fee,Gunner B lower fee,please contact vickie dvbrooks57gmailcom infor...,meet gunnerthis sweet retrieverlab mix 2 years...,Labrador Retriever,Labrador Retriever,Male,Male,0.905912


In [71]:
resultdef.to_csv('resultdef.cvs')

In [76]:
result_filtered.to_csv('result_filtered.cvs')